<a href="https://colab.research.google.com/github/yeop6134/Ai_humanities/blob/main/BERT%EA%B8%B0%EB%B0%98TopicModeling/KoBERTopic_%EB%B6%88%EC%9A%A9%EC%96%B4%EC%A0%9C%EA%B1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bertopic[visualization]

# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 100.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%pip install bertopic

In [ ]:
%pip install bertopic[visualization]

# 필요한 것들을 임포트

In [ ]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from bertopic import BERTopic

# 데이터 불러오기

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/인공지능인문학"

/content/drive/MyDrive/Colab Notebooks/인공지능인문학


In [ ]:
f = open("discrimination.txt")
documents = [line.strip() for line in open("discrimination.txt", encoding="utf-8").readlines()]
documents[:5]

['text',
 '남자들은 도대체 왜 그러는걸까?',
 '말하면서 침 개많이 튀길것 같아.',
 '머리가 멍청해서 못읽는거아니냐?',
 '"동유럽, 러시아 애들은 가난해도 사랑으로 결혼하는데 헬조선은 자본주의의 지옥 그 자체다"']

# Mecab과 SBERT를 이용한 Bertopic (형태소단위)

In [ ]:
# 불용어 사전 만들기
korean_stopwords_path = "./korean_stopwords.txt"

with open(korean_stopwords_path, encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.strip() for x in stopwords]

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
         if w not in stopwords: 
            result.append(w)

        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())

In [ ]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [ ]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=8,
                 top_n_words=10,
                 calculate_probabilities=True)

In [ ]:
topics, probs = model.fit_transform(documents)

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
model.visualize_distribution(probs[0])

In [ ]:
for i in range(0, 8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('ㅋㅋ', 0.04849572665753201), ('남자', 0.046716394343431965), ('장애', 0.03950820807305055), ('짱깨', 0.037027993370471926), ('사람', 0.03555059990296805), ('여자', 0.0351045263236232), ('전라도', 0.03044399513264479), ('한테', 0.028754131361663215), ('..', 0.028245546819483585), ('그렇', 0.028195924654195972)]
1 번째 토픽 : [('한국', 0.1390157686489593), ('조선족', 0.1301424217139193), ('중국', 0.1225136092678746), ('중국인', 0.07685785955529327), ('서울', 0.06650320792104829), ('일본', 0.06342654407454032), ('한국인', 0.049729494533604104), ('사람', 0.04803648873528577), ('인간', 0.045030694962412854), ('북한', 0.04290944831482688)]
2 번째 토픽 : [('ㅋㅋ', 0.060146595213494275), ('여자', 0.059819540996585506), ('ㅋㅋㅋ', 0.05160626397755651), ('는데', 0.04292438235079291), ('남자', 0.04277529934738112), ('으면', 0.03455432809560292), ('니까', 0.030293401618332168), ('어서', 0.0267136452257977), ('사람', 0.025039596322919883), ('라도', 0.02172405086299992)]
3 번째 토픽 : [('나와서', 0.06298062477848917), ('운증', 0.06093550350614863), ('후군', 0.0609355

## 명사단위 분석

In [ ]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.nouns(sent)

        words = [word for word in word_tokens if len(word) > 1]

        result = []
        for w in words: 
           if w not in stopwords: 
             result.append(w)
        return result

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics=8,
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(0, 8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('한국', 0.20806086282237649), ('조선족', 0.1903535553481317), ('중국', 0.1797911548794996), ('중국인', 0.11385519769130452), ('서울', 0.10478518214530089), ('사람', 0.07510634200491055), ('한국인', 0.07414237299845422), ('일본', 0.07029680675099377), ('인간', 0.06754776732868967), ('북한', 0.06395227660492361)]
1 번째 토픽 : [('여자', 0.10624965826370596), ('남자', 0.07928108001806301), ('사람', 0.05165687931879663), ('장애', 0.03347252315085146), ('흑인', 0.026901108492968832), ('얼굴', 0.025545932175702345), ('생각', 0.025446994520375923), ('결혼', 0.024547154344754504), ('나라', 0.021603524171006108), ('전라도', 0.021391046872083486)]
2 번째 토픽 : [('인간', 0.7198058955963492), ('집단', 0.43800742962767675), ('인물', 0.4254278284509251), ('조직', 0.21547627152675758), ('단체', 0.20909272764012213), ('지지', 0.18886195316781526), ('정당', 0.18886195316781526), ('방송인', 0.18317923251353743), ('정보', 0.17055288970350316), ('남성', 0.1428129352813598)]
3 번째 토픽 : [('발전', 0.21745151229055174), ('고향', 0.18317923251353743), ('말투', 0.1705528897035

In [ ]:
model.visualize_barchart()

## nr_topics="auto" 자동 토픽 축소 사용

In [ ]:
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics="auto",
                 top_n_words=10,
                 calculate_probabilities=True)
topics, probs = model.fit_transform(documents)
for i in range(0, 8):
  print(i,'번째 토픽 :', model.get_topic(i))

0 번째 토픽 : [('여자', 0.08691345268747172), ('남자', 0.0668564904508657), ('사람', 0.04464193447421608), ('장애', 0.02943123853462938), ('흑인', 0.02409625853435329), ('생각', 0.023119383188365993), ('얼굴', 0.022890439692048452), ('결혼', 0.02233170240814302), ('나라', 0.019937785922721232), ('전라도', 0.018575435363763382)]
1 번째 토픽 : [('한국', 0.24850285853457735), ('조선족', 0.19477251074331742), ('서울', 0.153277625869916), ('한국인', 0.09616188994036493), ('인간', 0.08974565046232089), ('인물', 0.07463357465372736), ('북한', 0.06983219301183073), ('사람', 0.06977838383330286), ('인천', 0.05673732103305903), ('집단', 0.055579665247548524)]
2 번째 토픽 : [('중국', 0.7085517767800339), ('중국인', 0.43216257778718026), ('유학', 0.10076463302092334), ('세계', 0.09851231447062983), ('이중국', 0.08489834883580806), ('적자', 0.08489834883580806), ('개국', 0.07970860533130353), ('짱꼴라', 0.07970860533130353), ('베트남', 0.07317659610161203), ('유전자', 0.07084765037690308)]
3 번째 토픽 : [('발전', 0.1947668002703832), ('글자', 0.18286091811299046), ('고향', 0.16253284498

In [ ]:
model.visualize_barchart()

In [ ]:
model.visualize_topics()

- 국가별로 토픽이 나뉜게 인상적이다. 1은 한국, 2는 중국, 5는 일본으로 나뉘었다.
- 그러나 토픽별로 문장 개수의 편향이 존재함을 알 수 있다.
  - '여자', '남자', '장애', '흑인' 등을 포함한 1번토픽의 크기가 압도적임을 알 수있다.
- 예상대로 국가별로 분류한 1, 2, 5번 토픽은 거리가 가깝다.
- 분석이 어려운 3, 4번 토픽의 거리가 가깝다.